In [1]:
import cv2
import keras
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import mutual_info_score, normalized_mutual_info_score
import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# list of networks to be studies
path1 = '/home/arash/Desktop/giessen/good/models/resnet50_org.h5'
path2 = '/home/arash/Desktop/giessen/good/models/resnet50_cv.h5'
path3 = '/home/arash/Desktop/giessen/good/models/resnet50_il.h5'
network1 = keras.models.load_model(path1)
network2 = keras.models.load_model(path2)
#network3 = keras.models.load_model(path3)

/home/arash/tensorflow3/lib/python3.5/site-packages/keras/engine/saving.py:270: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
# getting the weights, names, their difference for convolutional layers and depthwise conv
# keras.layers.core.Dense
def merge_weights(network_des, network_src, which_layers, which_kernels):
    for i, layer_name in enumerate(which_layers):
        layer_des = network_des.get_layer(layer_name).get_weights()
        layer_src = network_src.get_layer(layer_name).get_weights()
        for j in which_kernels[i]:
            layer_des[0][:,:,:,j] = layer_src[0][:,:,:,j]
            layer_des[1][j] = layer_src[1][j]
        network_des.get_layer(layer_name).set_weights(layer_des)
    return network_des

In [4]:
which_layers = ['res2a_branch2c', 'res3a_branch2c', 'res3c_branch2c', 'res4e_branch2c', 'res4f_branch2c']
which_kernels = [[204], [117], 
                 [27, 36, 109, 129, 138, 155, 190, 195, 260, 301, 368, 406, 456, 462, 478, 482, 493, 511],
                 [280, 422, 866], [831, 866]]

In [ ]:
network1_cv = merge_weights(network1, network2, which_layers, which_kernels)

In [18]:
n1 = network1.get_layer('res3c_branch2c')
n2 = network2.get_layer('res3c_branch2c')
print(np.sum(n1.get_weights()[1]-n2.get_weights()[1]))
print(n1.get_weights()[1].shape)
print(n1.get_weights()[0].shape)

-0.0014677551
(512,)
(1, 1, 128, 512)


In [23]:
n1 = network1.get_layer('res3c_branch2c')
n2 = network2.get_layer('res3c_branch2c')
print(np.sum(n1.get_weights()[1]-n2.get_weights()[1]))
print(n1.get_weights()[1].shape)
print(n1.get_weights()[0].shape)

-0.0014681607
(512,)
(1, 1, 128, 512)


In [26]:
n1 = network2.get_layer('res3c_branch2c')
n2 = network1_cv.get_layer('res3c_branch2c')
print(np.sum(n1.get_weights()[1][kernels]-n2.get_weights()[1][kernels]))
print(n1.get_weights()[1].shape)
print(n1.get_weights()[0].shape)

0.0
(512,)
(1, 1, 128, 512)


In [27]:
image_path = '/home/arash/Software/repositories/kernelphysiology/data/computervision/ilsvrc/ilsvrc2012/raw-data/validation/n03201208/ILSVRC2012_val_00000503.JPEG'
image = keras.preprocessing.image.load_img(image_path, target_size=(224,224))
x = keras.preprocessing.image.img_to_array(image)
x_pre = keras.applications.resnet50.preprocess_input(x.copy())
x_pre = np.expand_dims(x_pre, axis=0)

In [28]:
activities1 = network1_cv.predict(x_pre)

In [31]:
activities1_org = network1.predict(x_pre)

In [30]:
activities1.argmax()

532

In [32]:
activities1_org.argmax()

532

In [6]:
network1_cv.save('restnet50_corr50_merged.h5')